# Assignment 5
The script should not give an error. Any mistake will be consider as 0.




We want an excel file with all the available presidential elections in this webpuelta

This is an example for Presidencial 2021 - 2DA vuelta


In [12]:
!pip install selenium chromedriver_autoinstaller

Defaulting to user installation because normal site-packages is not writeable


In [13]:
pip install selenium webdriver-manager pandas tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [14]:
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.chrome.options import Options  
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time  
import pandas as pd  
from tqdm import tqdm  
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException  

# Setting ChromeDriver  
chrome_options = Options()  
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")  
chrome_options.add_argument("--disable-dev-shm-usage")  

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

## Surfing and extracting data:

In [15]:
# conecting to web site required  
driver.get("https://infogob.jne.gob.pe/Eleccion")  
time.sleep(10)  # Espera a que la página cargue completamente  

# going into presidencial elections
driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[1]/div').click()  
time.sleep(3)  
driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]').click()  
time.sleep(3)  
driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[2]/div').click()  
time.sleep(3)  

# elections list 
items = driver.find_elements(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[2]/div/div[2]')  
elecciones = items[0].text.split('\n')[1:]

## Extracting tables:

In [16]:
def extrae_tabla(driver):  #function for extract tables from presidential election results
    from io import StringIO  

    tabla = driver.find_element(By.XPATH, '//*[@id="CandidatosResultados"]/div/div[1]/div[2]/div[2]').get_attribute('innerHTML')  
    tabla_1 = StringIO(tabla)  
    data = pd.read_html(tabla_1)  
    return data[0]

## Next click

In [17]:
def boton_siguiente(driver, i):  
    try:  
        next_button = WebDriverWait(driver, 10).until(  
            EC.element_to_be_clickable((By.XPATH, f'//*[@id="CandidatosResultados"]/div/div[1]/div[2]/div[2]/div[4]/div[2]/ul/li[{i}]/a'))  
        )  
        next_button.click()  
        return True  
    except (TimeoutException, ElementClickInterceptedException):  
        return False  

## Iteration for each election:

In [18]:
all_data = []  
for x in tqdm(range(1, len(elecciones) + 1), desc='Progreso:'):  
    driver.get("https://infogob.jne.gob.pe/Eleccion")  
    time.sleep(3)  

    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[1]/div').click()  
    time.sleep(3)  
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]').click()  
    time.sleep(3)  
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[2]/div').click()  
    time.sleep(3)  

    y = x + 1  
    driver.find_element(By.XPATH, f'//*[@id="section"]/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[{y}]').click()  
    time.sleep(3)  

    driver.find_element(By.XPATH, '//*[@id="btnVerDatos"]/span').click()  
    time.sleep(3)  

    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[3]/div[1]/ul/li[2]/a').click()  
    time.sleep(3)  

    i = 3  
    while True:  
        aux = extrae_tabla(driver)  
        aux['Elecciones'] = elecciones[x - 1]  
        all_data.append(aux)  

        if boton_siguiente(driver, i):  
            i += 1  
        else:  
            break  

Progreso:: 100%|███████████████████████████████████████████████████████████████████████| 25/25 [12:48<00:00, 30.73s/it]


## Concatenating and saving data frames

In [19]:
# Concatenating data frames
data_elecciones = pd.concat(all_data, ignore_index=True)  
data_elecciones = data_elecciones[['Elecciones', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]  

# saving in excel file
data_elecciones.to_excel('Data_Elecciones_group_4.xlsx', sheet_name='Data', index=False)  

# quiting driver 
driver.quit()  